In [46]:
#ms = 20220410T0130  
#dt = datetime.fromtimestamp(ms/1000.0)
#dt

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from statistics import mean

api_key = "APCAF6OLCGOVGNUS"
url_interest = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=monthlytopics&apikey={api_key}'
url_cpi = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={api_key}'
url_coffee = f'https://www.alphavantage.co/query?function=COFFEE&interval=monthly&apikey={api_key}'

r1 = requests.get(url_interest)
r2 = requests.get(url_cpi)
r3 = requests.get(url_coffee)

interest_rate = r1.json()
cpi_us = r2.json()
coffee = r3.json()


print(interest_rate)
print(cpi_us)
print(coffee)

In [48]:
#printing the API-Data from Alpha Vantage
interest_rate = pd.DataFrame(interest_rate)
cpi_us = pd.DataFrame(cpi_us)
coffee = pd.DataFrame(coffee)

In [ ]:
#extracting Data from nested dict (interest rate) Alpha Vantage
values = [item["value"] for item in interest_rate["data"]]
dates = [item["date"] for item in interest_rate["data"]]
print(dates)
print(values)

In [50]:
#create the columns for interest rate
interest_rate["us_interest_rate_%"] = values
interest_rate["Date"] = dates

In [ ]:
interest_rate

In [52]:
interest_rate_clean = interest_rate.drop(columns = ["data", "name", "interval","unit"])    

In [ ]:
interest_rate_clean

In [ ]:
#extracting Data from nested dict (cpi) Alpha Vantage
values2 = [item["value"] for item in cpi_us["data"]]
dates2 = [item["date"] for item in cpi_us["data"]]
print(dates2)
print(values2)

In [55]:
cpi_us["cpi"] = values2
cpi_us["Date"] = dates2

In [ ]:
cpi_us

In [ ]:
cpi_us_clean = cpi_us.drop(columns = ["data", "name", "interval","unit"])
cpi_us_clean

In [ ]:
#extracting Data from nested dict (coffee) Alpha Vantage
values3 = [item["value"] for item in coffee["data"]]
dates3 = [item["date"] for item in coffee["data"]]
print(dates3)
print(values3)

In [59]:
coffee["coffee_price/cents_per_pound"] = values3
coffee["Date"] = dates3

In [ ]:
coffee

In [ ]:
coffee_clean = coffee.drop(columns = ["data", "name", "interval","unit"])

coffee_clean

coffee_clean
cpi_us_clean
interest_rate_clean

-> from API Alpha Vantage
next step -> merge them all

In [ ]:
path_btc = "./btc.csv"   #lokal path // need to be changed  "C:\Users\rk-admin\OneDrive\Desktop\DA_FT_FEB25_Ironhack-Bootcamp\week3\quest\btc.csv"
btc_kaggle = pd.read_csv(path_btc)
print (btc_kaggle.head())


In [ ]:
btc_kaggle_clean = btc_kaggle.drop(columns = ["Volume"])
cpi_us_clean = cpi_us.drop(columns = ["data", "name", "interval","unit"])
print (btc_kaggle_clean.head())



In [64]:
final_df = btc_kaggle_clean
final_df["Date"] = pd.to_datetime(btc_kaggle_clean["Date"])

high_mean = final_df.groupby(final_df["Date"].dt.to_period("M"))["High"].mean()
low_mean = final_df.groupby(final_df["Date"].dt.to_period("M"))["Low"].mean()
open = final_df.groupby(final_df["Date"].dt.to_period("M"))["Open"].first().reset_index()
close = final_df.groupby(final_df["Date"].dt.to_period("M"))["Close"].first().reset_index()

high_mean.index = high_mean.index.to_timestamp()
low_mean.index = low_mean.index.to_timestamp()

open["Date"] = open["Date"].dt.to_timestamp()
close["Date"] = close["Date"].dt.to_timestamp()

high_mean = high_mean.reset_index().rename(columns={"index": "Date"})
low_mean = low_mean.reset_index().rename(columns={"index": "Date"})


In [ ]:
print ("-------------high mean------------")
print(high_mean.head())
print ("----------------------------------")

print ("-------------low mean-------------")
print(low_mean.head())
print ("----------------------------------")

print ("-------------open btc-------------")
print(open.head())
print ("----------------------------------")

print ("-------------close btc-------------")
print(close.head())
print ("-----------------------------------")


In [ ]:

#merging all the parsed data into one table
btc_final = (
    high_mean
    .merge(low_mean, on="Date", how="inner")
    .merge(open, on="Date", how="inner")
    .merge(close, on="Date", how="inner")
)

#filtering by a limit date
limit_lower_date = pd.to_datetime("2010-08-01")
limit_upper_date = pd.to_datetime("2023-12-01")
btc_final = btc_final[btc_final["Date"] >= limit_lower_date]
btc_final = btc_final[btc_final["Date"] <= limit_upper_date]

#sorting by date so the highest date shows first
# btc_final = btc_final.sort_values(by="Date", ascending=False)

#rounding the numbers so it shows only one decimal
btc_final["High"] = btc_final["High"].round(1)
btc_final["Low"] = btc_final["Low"].round(1)

print(btc_final.head())

In [ ]:
coffee_clean["Date"] = pd.to_datetime(coffee_clean["Date"])

coffee_final = coffee_clean[coffee_clean["Date"] >= limit_lower_date]
coffee_final = coffee_final[coffee_final["Date"] <= limit_upper_date]

print(coffee_final)

In [ ]:
cpi_us_clean["Date"] = pd.to_datetime(cpi_us_clean["Date"])

cpi_us_final = cpi_us_clean[cpi_us_clean["Date"] >= limit_lower_date]
cpi_us_final = cpi_us_final[cpi_us_final["Date"] <= limit_upper_date]

print(cpi_us_final)

In [ ]:
interest_rate_clean["Date"] = pd.to_datetime(interest_rate_clean["Date"])

interest_rate_final = interest_rate_clean[interest_rate_clean["Date"] >= limit_lower_date]
interest_rate_final = interest_rate_final[interest_rate_final["Date"] <= limit_upper_date]

print(interest_rate_final)


In [ ]:
result = (
    btc_final
    .merge(cpi_us_final, on="Date", how="inner")
    .merge(coffee_final, on="Date", how="inner")
    .merge(interest_rate_final, on="Date", how="inner")
)

print(result)

In [ ]:
path_coffee_btc = "./gtrends_btc_coffee_xls.xlsx"
coffee_btc = pd.read_excel(path_coffee_btc)

coffee_btc_clean = coffee_btc.rename(columns={"month": "Date"})

coffee_btc_clean["Date"] = pd.to_datetime((coffee_btc_clean["Date"]))

print(coffee_btc_clean.head())

In [ ]:
coffee_btc_final = coffee_btc_clean[coffee_btc_clean["Date"] >= limit_lower_date]
coffee_btc_final = coffee_btc_final[coffee_btc_final["Date"] <= limit_upper_date]

coffee_btc_final = coffee_btc_final.sort_values(by="Date", ascending=False)

print(coffee_btc_final)

In [ ]:
result = pd.merge(result, coffee_btc_final, on="Date", how="inner")

result = result.sort_values(by="Date", ascending=False)

print(result.head())

In [ ]:
result["monthly_mean_btc"] = (result["High"] + result["Low"]) / 2


# Reordering for comparing the difference with the one before
result = result.sort_values(by="Date", ascending=True)

result["change_btc%"] = result["Open"].pct_change() *100
result["change_btc%"] = result["change_btc%"].fillna(0)

result = result.sort_values(by="Date", ascending=False)

print(result.head())

In [ ]:
result = result.sort_values(by="Date", ascending=True)

result["change_trends_btc%"] = result["google_trends_btc"].pct_change() *100
result["change_trends_btc%"] = result["change_trends_btc%"].fillna(0)

result = result.sort_values(by="Date", ascending=False)

print(result.head())

In [ ]:
result = result.sort_values(by="Date", ascending=True)

# coffee_price/cents_per_pound is a string for some reason so i convert it into a number
result["coffee_price/cents_per_pound"] = pd.to_numeric(result["coffee_price/cents_per_pound"])

result["change_coffee_price%"] = result["coffee_price/cents_per_pound"].pct_change() *100
result["change_coffee_price%"] = result["change_coffee_price%"].fillna(0)

result = result.sort_values(by="Date", ascending=False)

print(result.head())

In [ ]:
result = result.sort_values(by="Date", ascending=True)

result["change_trends_coffee%"] = result["google_trends_coffee"].pct_change() *100
result["change_trends_coffee%"] = result["change_trends_coffee%"].fillna(0)

result = result.sort_values(by="Date", ascending=False)
print(result.head())


In [ ]:
import matplotlib.pyplot as plt

# Ordering date so it prints the oldest at the left
result = result.sort_values(by="Date", ascending=True)


# Create shape
plt.figure(figsize=(12, 6))

# Coffee graph
plt.plot(result["Date"], result["change_coffee_price%"], marker="o", linestyle="-", color="brown", label="Coffee variation%")

# Btc graph
plt.plot(result["Date"], result["change_btc%"], marker="s", linestyle="--", color="blue", label="Btc variation%")

# design
plt.xlabel("Date")
plt.ylabel("Price Change%")
plt.title("Comparison of the monthly change of prices")
plt.xticks(rotation=45)
plt.legend()
plt.grid()

# cutting top values so we can see correctly
plt.ylim(-50, 100)
plt.xlim(pd.to_datetime('2010-08-01'), result["Date"].max())

# show graph
plt.show()



In [ ]:

# Dispersion graph
plt.figure(figsize=(10, 6))
scatter = plt.scatter(result["change_trends_btc%"], result["change_btc%"], c=result["change_trends_coffee%"], cmap='viridis', s=100)

# Design 
plt.title('Relationship between BTC Change, BTC Trends, and Coffee Trends', fontsize=14)
plt.xlabel('Change in Google Trends BTC (%)', fontsize=12)
plt.ylabel('change in BTC (%)', fontsize=12)

# Color (for visualitation `change_trends_coffee%`)
plt.colorbar(scatter, label='Change in Google Trends Coffee (%)')

plt.grid(True)
plt.show()

In [ ]:
# Btc variation graph
plt.plot(result["Date"], result["change_btc%"], marker="s", linestyle="-", color="blue", label="Btc variation%")

# Google Trends BTC variation graph
plt.plot(result["Date"], result["change_trends_btc%"], marker="o", linestyle=":", color="brown", label="Google Trends BTC variation%")

# design
plt.xlabel("Date")
plt.ylabel("Variation (%)")
plt.title("Comparison of the monthly change in BTC and Google Trends BTC")
plt.xticks(rotation=45)
plt.legend()
plt.grid()

# cutting top values so we can see correctly
plt.ylim(-50, 100)
plt.xlim(pd.to_datetime('2010-08-01'), result["Date"].max())

# show graph
plt.show()

cpi / btc
interest rate / btc
